# **Lmfit Implementation**

In [ ]:
!pip install lmfit
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()

from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, PandasUDFType

spark = SparkSession.builder \
        .master("local[*]") \
        .appName("Spark RDD Application") \
        .config(conf = SparkConf()) \
        .getOrCreate()

spark

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
from scipy.optimize import curve_fit, minimize
import pandas as pd
import math

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructType, StructField, StringType,BooleanType
from pyspark.sql import SparkSession
import collections
from pyspark.sql.functions import col


from lmfit import Parameters , minimize, Minimizer
from lmfit import Parameters , minimize, Minimizer

# -*- coding: utf-8 -*-
import numpy as np
from scipy.optimize import curve_fit, minimize
import pandas as pd
import collections
from pyspark.sql.functions import col, lit, when, greatest, least, exp, sqrt, abs
from lmfit import Parameters , minimize, Minimizer
from pyspark.sql import functions as f

In [ ]:
#Creating dataframe
name_1 = np.linspace(0.0, 5., 2000000)

name_2 = np.linspace(0.0, 10., 2000000)

name_3 = np.linspace(0.0, 15., 2000000)

name_4 = np.linspace(0.0, 20., 2000000)

name_5 = np.linspace(0.0, 25., 2000000)

name_6 = np.linspace(0.0, 3., 2000000)

name_7 = np.linspace(0.0, 6., 2000000)

name_8 = np.linspace(0.0, 9., 2000000)

name_9 = np.linspace(0.0, 5., 2000000)

name_11 = np.linspace(0.0, 2., 2000000)

name_12 = np.linspace(0.0, 4., 2000000)

Y_act  = np.linspace(0.0, 10., 2000000)
wgt  = np.linspace(0.0, 3., 2000000)

group = ['group_name' for i in range(2000000)]

df_data= pd.DataFrame({'name_1' : name_1 , 'name_2':name_2 , 'name_3':name_3 ,'name_4':name_4,'name_5':name_5,'name_6':name_6,
                       'name_7':name_7,'name_8':name_8,'name_9':name_9,'name_11':name_11,'name_12':name_12,
                       'Y_act':Y_act ,'wgt':wgt,'group':group })
df_data_1 = spark.createDataFrame(df_data)

/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [ ]:
name_10 = []
string = ['f_1', 'f_2', 'f_3' , 'f_4' ,'f_5']
for strs in string:
    if strs == 'f_1':
        name_10.extend(['f_1' for i in range(4)])
    elif strs == 'f_2':
        name_10.extend(['f_2' for i in range(4)])
    elif strs == 'f_3':
        name_10.extend(['f_3' for i in range(4)])
    elif strs == 'f_4':
        name_10.extend(['f_4' for i in range(4)])
    else:
        name_10.extend(['f_5' for i in range(4)])
print(len(name_10))

#################################################
import random
list1 = [i for i in range(1 , 3)]
list2 = [i for i in range(3 , 5)]
list3 = [i for i in range(2 , 4)]

name_A = [random.choice(list1) for i in range(20)]
name_B = [random.choice(list2) for i in range(20)]
name_C = [random.choice(list3) for i in range(20)]

#################################################

PARAM_1 = ['ABC_'+str(i) for i in range(20)]
value_1 = np.linspace(0.0, 25., 20)

PARAM_2 = ['XYZ_'+str(i) for i in range(20)]
value_2 = np.linspace(0.0, 50., 20)

#################################################
key = ['X'+str(i) for i in range(20)]

################################################
df_param= pd.DataFrame({'PARAM_1' : PARAM_1 ,'value_1' : value_1 , 'PARAM_2':PARAM_2 ,'value_2' : value_2 , 'name_10':name_10 ,
                        'name_A':name_A,'name_B':name_B,'name_C':name_C,'key':key})
df_param_1 = spark.createDataFrame(df_param)

20


/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [ ]:
def model_param(df):
    import collections
    dict_1 = {df['key'][i]:df['name_10'][i] for i in range(20)}
    dict_2 = {df['key'][i]:[df['name_A'][i],df['name_B'][i],df['name_C'][i]] for i in range(20)}
    dict_PM_1 = {df['PARAM_1'][i]:df['value_1'][i] for i in range(20)}
    dict_PM_2 = {df['PARAM_2'][i]:df['value_2'][i] for i in range(20)}
    dict_3 = {df['PARAM_1'][i]:df['key'][i] for i in range(20)}
    dict_4 = {df['PARAM_2'][i]:df['key'][i] for i in range(20)}

    pm_AY = Parameters()
    pm_EY = Parameters()
    for i in range(20):
        pm_AY.add('ABC_'+str(i), value =dict_PM_1['ABC_'+str(i)])
        pm_EY.add('XYZ_'+str(i), value =dict_PM_2['XYZ_'+str(i)])

    PM_col = collections.namedtuple('all_col' , ['dict_1','dict_2','pm_AY','pm_EY','dict_3','dict_4'])
    PM = PM_col(dict_1=dict_1,dict_2=dict_2,pm_AY=pm_AY,pm_EY=pm_EY,dict_3=dict_3,dict_4=dict_4)

    return PM

In [ ]:
PM = model_param(df_param)

In [ ]:
def func_plin(f , x , a, b, c):
    if f == 'f_1':
        return np.maximum(0 , np.minimum(1 , (x-a)/(b-a)))
    elif f == 'f_2':
        return np.maximum(0 , np.minimum(1 , (x-a)/(b-c)))
    elif f == 'f_3':
        return np.maximum(0 , np.minimum(1 , (x-a)*(b-a)))
    elif f == 'f_4':
        return np.maximum(0 , np.minimum(1 , (x-a)*(b-c)))
    elif f == 'f_5':
        return np.maximum(0 , np.minimum(1 , (x-a)*(b-c)))
    else:
        return (1)

##########################################################

def model_func_fit(pm_EY_EST , PM  ,funcid , submod, df):
    y_mod = model_func(pm_EY_EST , PM  ,funcid , submod, df)
    er = y_mod - df['Y_act']
    er = er * np.sqrt(np.abs(df['wgt']))
    return er

############################################################

def model_func(pm_EY_EST , PM ,funcid , submod , df):
    if submod == 'TOT':
        y_mod = model_func_tot(pm_EY_EST , PM , df ,funcid )
    elif submod == 'PRX':
        y_mod = model_func_prx(pm_EY_EST , PM , df ,funcid )
    else:
        y_mod = model_func_cst(pm_EY_EST , PM , df ,funcid )
    return  y_mod
##############################################################

def model_func_pmilvl(pm_EY_EST , PM , df ,funcid):
    pm_2= PM.dict_1
    dict_F = PM.dict_2
    pm_AY_1 = PM.pm_AY
    #pm_EY_1 = pm_EY_EST + pm_AY_1
    pm_EY_1 = pm_AY_1 + pm_EY_EST

    pmilvl = 0
    pmilvl = pmilvl + pm_EY_1['XYZ_0']+ (df['name_1']+ pm_EY_1['ABC_0']) +pm_EY_1['XYZ_5'] + pm_EY_1['ABC_5']
    pmilvl = pmilvl + pm_EY_1['XYZ_1']- (df['name_2']/ pm_EY_1['ABC_1']) +pm_EY_1['XYZ_6'] + pm_EY_1['ABC_6']
    pmilvl = pmilvl + pm_EY_1['XYZ_2']- (df['name_3']/ pm_EY_1['ABC_2']) +pm_EY_1['XYZ_7']+pm_EY_1['ABC_7']
    pmilvl = pmilvl + pm_EY_1['XYZ_3']- (df['name_4']/ pm_EY_1['ABC_3']) +pm_EY_1['XYZ_8']+ pm_EY_1['ABC_8']
    pmilvl = pmilvl + pm_EY_1['XYZ_4']- (df['name_5']/ pm_EY_1['ABC_4']) +pm_EY_1['XYZ_9']+ pm_EY_1['ABC_9']

    pmilvl_1 = 0
    if funcid == 1:
        for i in range(0 , 10):
            pmilvl_1 = pmilvl_1 + func_plin(pm_2['X'+str(i)] ,df['name_12']  ,dict_F['X'+str(i)][0], dict_F['X'+str(i)][1], dict_F['X'+str(i)][2])


    return  pmilvl +  pmilvl_1


####################################################################

def model_func_tot(pm_EY_EST , PM , df ,funcid):
    pmilvl = model_func_pmilvl(pm_EY_EST , PM , df ,funcid)

    pm_2= PM.dict_1
    dict_F = PM.dict_2
    pm_AY_1 = PM.pm_AY
    #pm_EY_1 = pm_EY_EST + pm_AY_1
    pm_EY_1 = pm_AY_1 + pm_EY_EST

    tot = 0
    tot = tot + pm_EY_1['XYZ_10']- (df['name_6']+ pm_EY_1['ABC_15']) +pm_EY_1['ABC_10'] + pm_EY_1['XYZ_15']
    tot = tot + pm_EY_1['XYZ_11']- (df['name_7']/ pm_EY_1['ABC_16']) +pm_EY_1['ABC_11'] + pm_EY_1['XYZ_16']
    tot = tot + pm_EY_1['XYZ_12']- (df['name_8']/ pm_EY_1['ABC_17']) +pm_EY_1['ABC_12'] + pm_EY_1['XYZ_17']
    tot = tot + pm_EY_1['XYZ_13']- (df['name_9']/ pm_EY_1['ABC_18']) +pm_EY_1['ABC_13'] + pm_EY_1['XYZ_18']
    tot = tot + pm_EY_1['XYZ_14']- (df['name_11']/ pm_EY_1['ABC_19']) +pm_EY_1['ABC_14'] + pm_EY_1['XYZ_19']

    tot_1 = 0
    if funcid == 1:
        for i in range(0 , 10):
            tot_1 = tot_1 + func_plin(pm_2['X'+str(i)] ,df['name_12']  ,dict_F['X'+str(i)][0], dict_F['X'+str(i)][1], dict_F['X'+str(i)][2])

    elif  funcid == 2:
        for i in range(10 , 20):
            tot_1 = tot_1 + func_plin(pm_2['X'+str(i)] ,df['name_12']  ,dict_F['X'+str(i)][0], dict_F['X'+str(i)][1], dict_F['X'+str(i)][2])

    return  pmilvl + tot +  np.exp(tot_1)

########################################################

def model_func_prx(pm_EY_EST , PM , df ,funcid):
    pm_1= PM.dict_1
    dict_f = PM.dict_2
    pm_AY = PM.pm_AY
    pm_EY = pm_EY_EST + pm_AY

    prx = 0
    prx = prx + pm_EY['XYZ_0']+ (df['name_1']+ pm_EY['ABC_0']) +pm_EY['XYZ_5'] + pm_EY['ABC_5']
    prx = prx + pm_EY['XYZ_1']- (df['name_2']/ pm_EY['ABC_1']) +pm_EY['XYZ_6'] + pm_EY['ABC_6']
    prx = prx + pm_EY['XYZ_2']- (df['name_3']/ pm_EY['ABC_2']) +pm_EY['XYZ_7']+ pm_EY['ABC_7']
    prx = prx + pm_EY['XYZ_3']- (df['name_4']/ pm_EY['ABC_3']) +pm_EY['XYZ_8']+ pm_EY['ABC_8']
    prx = prx + pm_EY['XYZ_4']- (df['name_5']/ pm_EY['ABC_4']) +pm_EY['XYZ_9']+ pm_EY['ABC_9']
    prx = prx + pm_EY['XYZ_10']- (df['name_6']/ pm_EY['ABC_15']) +pm_EY['ABC_10'] + pm_EY['XYZ_15']
    prx = prx + pm_EY['XYZ_11']- (df['name_7']/ pm_EY['ABC_16']) +pm_EY['ABC_11'] + pm_EY['XYZ_16']
    prx = prx + pm_EY['XYZ_12']- (df['name_8']/ pm_EY['ABC_17']) +pm_EY['ABC_12'] + pm_EY['XYZ_17']
    prx = prx + pm_EY['XYZ_13']- (df['name_9']/ pm_EY['ABC_18']) +pm_EY['ABC_13'] + pm_EY['XYZ_18']
    prx = prx + pm_EY['XYZ_14']- (df['name_11']/ pm_EY['ABC_19']) +pm_EY['ABC_14'] + pm_EY['XYZ_19']

    prx_1 = 0
    if funcid == 1:
        for i in range(0 , 10):
            prx_1 = prx_1 + func_plin(pm_1['X'+str(i)] ,df['name_12']  ,dict_f['X'+str(i)][0], dict_f['X'+str(i)][1], dict_f['X'+str(i)][2])

    elif  funcid == 2:
        for i in range(10 , 20):
            prx_1 = prx_1 + func_plin(pm_1['X'+str(i)] ,df['name_12']  ,dict_f['X'+str(i)][0], dict_f['X'+str(i)][1], dict_f['X'+str(i)][2])

    return  np.exp(prx + prx_1)

#######################################################################

def model_func_cst(pm_EY_EST , PM , df ,funcid):
    pm_1= PM.dict_1
    dict_f = PM.dict_2
    pm_AY = PM.pm_AY
    pm_EY = pm_EY_EST + pm_AY

    cst = 0
    cst = cst + pm_EY['XYZ_0']+ (df['name_1']+ pm_EY['ABC_0']) /pm_EY['XYZ_5'] - pm_EY['ABC_5']
    cst = cst + pm_EY['XYZ_1']+ (df['name_2']/ pm_EY['ABC_1']) /pm_EY['XYZ_6'] - pm_EY['ABC_6']
    cst = cst + pm_EY['XYZ_2']+ (df['name_3']/ pm_EY['ABC_2']) /pm_EY['XYZ_7']- pm_EY['ABC_7']
    cst = cst + pm_EY['XYZ_3']+ (df['name_4']/ pm_EY['ABC_3']) /pm_EY['XYZ_8']- pm_EY['ABC_8']
    cst = cst + pm_EY['XYZ_4']+ (df['name_5']/ pm_EY['ABC_4']) /pm_EY['XYZ_9']- pm_EY['ABC_9']
    cst = cst + pm_EY['XYZ_10']+ (df['name_6']/ pm_EY_1['ABC_15']) /pm_EY['ABC_10'] + pm_EY['XYZ_15']
    cst = cst + pm_EY['XYZ_11']+ (df['name_7']/ pm_EY_1['ABC_16']) /pm_EY['ABC_11'] + pm_EY['XYZ_16']
    cst = cst + pm_EY['XYZ_12']+ (df['name_8']/ pm_EY_1['ABC_17']) /pm_EY['ABC_12'] + pm_EY['XYZ_17']
    cst = cst + pm_EY['XYZ_13']+ (df['name_9']/ pm_EY_1['ABC_18']) /pm_EY['ABC_13'] + pm_EY['XYZ_18']
    cst = cst + pm_EY['XYZ_14']+ (df['name_11']/ pm_EY_1['ABC_19']) /pm_EY['ABC_14'] + pm_EY['XYZ_19']

    cst_1 = 0
    if funcid == 1:
        for i in range(0 , 10):
            cst_1 = cst_1 + func_plin(pm_1['X'+str(i)] ,df['name_12']  ,dict_f['X'+str(i)][0], dict_f['X'+str(i)][1], dict_f['X'+str(i)][2])

    elif  funcid == 2:
        for i in range(10 , 20):
            cst_1 = cst_1 + func_plin(pm_1['X'+str(i)] ,df['name_12']  ,dict_f['X'+str(i)][0], dict_f['X'+str(i)][1], dict_f['X'+str(i)][2])

    return  np.exp(cst + cst_1)

# **Training Lmfit model with python function**

In [ ]:
import datetime
start_time = datetime.datetime.now()
print ("Time start time is  : ", end = "")
print (start_time)

#############################################

pm_EY_EST = PM.pm_EY
funcid = 1
submod = 'TOT'
out = Minimizer(model_func_fit, pm_EY_EST  ,fcn_args = ( PM  ,funcid , submod,df_data,))
out = out.minimize()

###########################################
end_time = datetime.datetime.now()
print ("Time end time is  : ", end = "")
print (end_time)

Time start time is  : 2023-12-19 08:26:41.828867
Time end time is  : 2023-12-19 08:27:16.467235


In [ ]:
out

# **Training Lmfit model with pyspark wrapped framework @pandas_udf**

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType,BooleanType
schema = StructType([ \
        StructField("PM_NM",StringType(),True), \
        StructField("Estimate",DoubleType(),True), \
        StructField("Std. Error",DoubleType(),True), \
        StructField("min",DoubleType(),True), \
        StructField("max",DoubleType(),True), \
        StructField("vary",BooleanType(),True),
      ])

##########################################################

@pandas_udf(schema , PandasUDFType.GROUPED_MAP)
def minimizer(df):
    #out = minimize(residual, fit_params ,args = (df,))
    out = Minimizer(model_func_fit, pm_EY_EST  ,fcn_args = ( PM  ,funcid , submod,df,))
    out = out.minimize()
    result = out.params
    a = []
    b = []
    c = []
    d = []
    e = []
    f = []
    for p in result:
        a.append(p)
        b.append(result[p].value)
        c.append(result[p].stderr)
        d.append(result[p].min)
        e.append(result[p].max)
        f.append(result[p].vary)
    return pd.DataFrame({'PM_NM':a ,'Estimate':b , 'Std. Error':c , 'min' :d ,'max':e , 'vary':f })

In [ ]:
import datetime
group_column = 'group'
start_time = datetime.datetime.now()
print ("Time start time is  : ", end = "")
print (start_time)

#####################
test = df_data_1.groupby(group_column).apply(minimizer)


#####################

end_time = datetime.datetime.now()
print ("Time start time is  : ", end = "")
print (end_time)

Time start time is  : 2023-12-19 08:01:43.952002
Time start time is  : 2023-12-19 08:01:44.245742


/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/sql/pandas/group_ops.py:81: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [ ]:
import datetime
start_time = datetime.datetime.now()
print ("Time start time is  : ", end = "")
print (start_time)

#####################

test.show()

#####################

end_time = datetime.datetime.now()
print ("Time start time is  : ", end = "")
print (end_time)

Time start time is  : 2023-12-19 08:01:49.463911
+------+-------------------+--------------------+---------+--------+----+
| PM_NM|           Estimate|          Std. Error|      min|     max|vary|
+------+-------------------+--------------------+---------+--------+----+
| XYZ_0|-12111.047299247506|3.994877086785617E11|-Infinity|Infinity|true|
| XYZ_1| 1404.8592734972308|3.607017426750385E10|-Infinity|Infinity|true|
| XYZ_2| -2675.931565807987|1.274998114858630...|-Infinity|Infinity|true|
| XYZ_3|  930.7265786599944|1.377419360882776...|-Infinity|Infinity|true|
| XYZ_4|  1307.372254605729|1.940501834139460...|-Infinity|Infinity|true|
| XYZ_5|-1651.5607924331475|2.396394012196671...|-Infinity|Infinity|true|
| XYZ_6|-107.31803454703649|1.575392961205374...|-Infinity|Infinity|true|
| XYZ_7|  2451.503058567756|3.578239697833732E11|-Infinity|Infinity|true|
| XYZ_8| -500.2773648657934|7.432040380794957E10|-Infinity|Infinity|true|
| XYZ_9| -673.2018517269822|1.001882390018745...|-Infinity|Infi

# **Training Lmfit model after converting python function into pyspark and embedding those function inside python Lmfit function**

In [ ]:
def func_plin(f, df,col,  a, b, c):
  if f == 'f_1':
    return df.withColumn('values' , greatest(lit(0) , (least(lit(1) , (df[col] - lit(a + 0)) / (lit(b + 0) - lit(a + 0)) ))))

  elif f == 'f_2':
    return df.withColumn('values' , greatest(lit(0) , (least(lit(1) , (df[col] - lit(a + 0)) / (lit(b + 0) - lit(c + 0)) ))))
  elif f == 'f_3':
    return df.withColumn('values' , greatest(lit(0) , (least(lit(1) , (df[col] - lit(a + 0)) * (lit(b + 0) - lit(a + 0)) ))))
  elif f == 'f_4':
    return df.withColumn('values' , greatest(lit(0) , (least(lit(1) , (df[col] - lit(a + 0)) * (lit(b + 0) - lit(c + 0)) ))))
  elif f == 'f_5':
    return df.withColumn('values' , greatest(lit(0) , (least(lit(1) , (df[col] - lit(a + 0)) * (lit(b + 0) - lit(c + 0)) ))))
  else:
    return df.withColumn('values' , lit(1))

############################################################################################

def model_func_pmilvl(pm_EY_EST , PM , df ,funcid):
    pm_2= PM.dict_1
    dict_F = PM.dict_2
    pm_AY_1 = PM.pm_AY
    #pm_EY_1 = pm_EY_EST + pm_AY_1
    pm_EY_1 = pm_AY_1 + pm_EY_EST

    pmilvl = 0
    pmilvl = df.withColumn('result' , pmilvl + lit(pm_EY_1['XYZ_0'] + 0) + (col('name_1')+ lit(pm_EY_1['ABC_0'] + 0)) +lit(pm_EY_1['XYZ_5'] + 0) + lit(pm_EY_1['ABC_5'] + 0))
    pmilvl = pmilvl.withColumn('result' , col('result') - lit(pm_EY_1['XYZ_1'] + 0) / (col('name_2')+ lit(pm_EY_1['ABC_1'] + 0)) +lit(pm_EY_1['XYZ_6'] + 0) + lit(pm_EY_1['ABC_6'] + 0))
    pmilvl = pmilvl.withColumn('result' , col('result') - lit(pm_EY_1['XYZ_2'] + 0) / (col('name_3')+ lit(pm_EY_1['ABC_2'] + 0)) +lit(pm_EY_1['XYZ_7'] + 0) + lit(pm_EY_1['ABC_7'] + 0))
    pmilvl = pmilvl.withColumn('result' , col('result') - lit(pm_EY_1['XYZ_3'] + 0) / (col('name_4')+ lit(pm_EY_1['ABC_3'] + 0)) +lit(pm_EY_1['XYZ_8'] + 0) + lit(pm_EY_1['ABC_8'] + 0))
    pmilvl = pmilvl.withColumn('result' , col('result') - lit(pm_EY_1['XYZ_4'] + 0) / (col('name_5')+ lit(pm_EY_1['ABC_4'] + 0)) +lit(pm_EY_1['XYZ_9'] + 0) + lit(pm_EY_1['ABC_9'] + 0))

    if funcid == 1:
      pmilvl_1 = pmilvl.withColumn('result1' , lit(0))

      for i in range(0 , 10):
        params = Parameters()
        params.add('a', value=float(dict_F['X'+str(i)][0]))
        params.add('b', value=float(dict_F['X'+str(i)][1]))
        params.add('c', value=float(dict_F['X'+str(i)][2]))
        pmilvl_1 = func_plin(pm_2['X'+str(i)], pmilvl_1,'name_12',  params['a'], params['b'], params['c'])
        pmilvl_1 = pmilvl_1.withColumn('result1' , f.col('result1') + pmilvl_1['values'])
        pmilvl_1 = pmilvl_1.withColumn('result' , f.col('result') + pmilvl_1['result1'])

    return pmilvl_1

#######################################################################################

def model_func_tot(pm_EY_EST , PM , df ,funcid):
    df = model_func_pmilvl(pm_EY_EST , PM , df ,funcid)
    df = df.withColumn('mod_pmilvl' , col('result')).drop('result')

    pm_2= PM.dict_1
    dict_F = PM.dict_2
    pm_AY_1 = PM.pm_AY
    #pm_EY_1 = pm_EY_EST + pm_AY_1
    pm_EY_1 = pm_AY_1 + pm_EY_EST

    tot = 0
    tot = df.withColumn('result' , tot + lit(pm_EY_1['XYZ_10'] + 0) - (col('name_6')+ lit(pm_EY_1['ABC_15'] + 0)) +lit(pm_EY_1['ABC_10'] + 0) + lit(pm_EY_1['XYZ_15'] + 0))
    tot = tot.withColumn('result' , col('result') + lit(pm_EY_1['XYZ_11'] + 0) - (col('name_7')/ lit(pm_EY_1['ABC_16'] + 0)) +lit(pm_EY_1['ABC_11'] + 0) + lit(pm_EY_1['XYZ_16'] + 0))
    tot = tot.withColumn('result' , col('result') + lit(pm_EY_1['XYZ_12'] + 0) - (col('name_8')/ lit(pm_EY_1['ABC_17'] + 0)) +lit(pm_EY_1['ABC_12'] + 0) + lit(pm_EY_1['XYZ_17'] + 0))
    tot = tot.withColumn('result' , col('result') + lit(pm_EY_1['XYZ_13'] + 0) - (col('name_9')/ lit(pm_EY_1['ABC_18'] + 0)) +lit(pm_EY_1['ABC_13'] + 0) + lit(pm_EY_1['XYZ_18'] + 0))
    tot = tot.withColumn('result' , col('result') + lit(pm_EY_1['XYZ_14'] + 0) - (col('name_11')/ lit(pm_EY_1['ABC_19'] + 0)) +lit(pm_EY_1['ABC_14'] + 0) + lit(pm_EY_1['XYZ_19'] + 0))

    if funcid == 1:
      tot_1 = tot.withColumn('result1' , lit(0))
      for i in range(0 , 10):
        params = Parameters()
        params.add('a', value=float(dict_F['X'+str(i)][0]))
        params.add('b', value=float(dict_F['X'+str(i)][1]))
        params.add('c', value=float(dict_F['X'+str(i)][2]))
        tot_1 = func_plin(pm_2['X'+str(i)] ,tot_1 ,'name_12' ,params['a'], params['b'], params['c'])
        tot_1 = tot_1.withColumn('result1' , f.col('result1') + tot_1['values'])

    elif  funcid == 2:
      for i in range(10 , 20):
        params = Parameters()
        params.add('a', value=float(dict_F['X'+str(i)][0]))
        params.add('b', value=float(dict_F['X'+str(i)][1]))
        params.add('c', value=float(dict_F['X'+str(i)][2]))
        tot_1 = func_plin(pm_2['X'+str(i)] ,tot_1 , 'name_12' ,params['a'], params['b'], params['c'])
        tot_1 = tot_1.withColumn('result1' , f.col('result1') + tot_1['values'])


    return  tot_1.withColumn('result' , f.col('mod_pmilvl') + f.col('result') + exp(col('result1')))

#####################################################################################################

def model_func_prx(pm_EY_EST , PM , df ,funcid):
    pm_1= PM.dict_1
    dict_f = PM.dict_2
    pm_AY = PM.pm_AY
    pm_EY = pm_EY_EST + pm_AY

    prx = 0
    prx = df.withColumn('result' , prx + lit(pm_EY['XYZ_0'] + 0) + (col('name_1')+ lit(pm_EY['ABC_0'] + 0)) +lit(pm_EY['XYZ_5'] + 0) + lit(pm_EY['ABC_5'] + 0))
    prx = prx.withColumn('result' , col('result') + lit(pm_EY['XYZ_1'] + 0) - (col('name_2')/ lit(pm_EY['ABC_1'] + 0)) +lit(pm_EY['XYZ_6'] + 0) + lit(pm_EY['ABC_6'] + 0))
    prx = prx.withColumn('result' , col('result') + lit(pm_EY['XYZ_2'] + 0) - (col('name_3')/ lit(pm_EY['ABC_2'] + 0)) +lit(pm_EY['XYZ_7'] + 0) + lit(pm_EY['ABC_7'] + 0))
    prx = prx.withColumn('result' , col('result') + lit(pm_EY['XYZ_3'] + 0) - (col('name_4')/ lit(pm_EY['ABC_3'] + 0)) +lit(pm_EY['XYZ_8'] + 0) + lit(pm_EY['ABC_8'] + 0))
    prx = prx.withColumn('result' , col('result') + lit(pm_EY['XYZ_4'] + 0) - (col('name_5')/ lit(pm_EY['ABC_4'] + 0)) +lit(pm_EY['XYZ_9'] + 0) + lit(pm_EY['ABC_9'] + 0))
    prx = prx.withColumn('result' , col('result') + lit(pm_EY['XYZ_10'] + 0) - (col('name_6')/ lit(pm_EY['ABC_15'] + 0)) +lit(pm_EY['ABC_10'] + 0) + lit(pm_EY['XYZ_15'] + 0))
    prx = prx.withColumn('result' , col('result') + lit(pm_EY['XYZ_11'] + 0) - (col('name_7')/ lit(pm_EY['ABC_16'] + 0)) +lit(pm_EY['ABC_11'] + 0) + lit(pm_EY['XYZ_16'] + 0))
    prx = prx.withColumn('result' , col('result') + lit(pm_EY['XYZ_12'] + 0) - (col('name_8')/ lit(pm_EY['ABC_17'] + 0)) +lit(pm_EY['ABC_12'] + 0) + lit(pm_EY['XYZ_17'] + 0))
    prx = prx.withColumn('result' , col('result') + lit(pm_EY['XYZ_13'] + 0) - (col('name_9')/ lit(pm_EY['ABC_18'] + 0)) +lit(pm_EY['ABC_13'] + 0) + lit(pm_EY['XYZ_18'] + 0))
    prx = prx.withColumn('result' , col('result') + lit(pm_EY['XYZ_14'] + 0) - (col('name_11')/ lit(pm_EY['ABC_19'] + 0)) +lit(pm_EY['ABC_14'] + 0) + lit(pm_EY['XYZ_19'] + 0))


    if funcid == 1:
      prx_1 = prx.withColumn('result1' , lit(0))
      for i in range(0 , 10):
        params = Parameters()
        params.add('a', value=float(dict_F['X'+str(i)][0]))
        params.add('b', value=float(dict_F['X'+str(i)][1]))
        params.add('c', value=float(dict_F['X'+str(i)][2]))
        prx_1 = func_plin(pm_2['X'+str(i)] ,prx_1 ,'name_12' ,params['a'], params['b'], params['c'])
        prx_1 = prx_1.withColumn('result1' , f.col('result1') + prx_1['values'])

    elif  funcid == 2:
        for i in range(10 , 20):
            params = Parameters()
            params.add('a', value=float(dict_F['X'+str(i)][0]))
            params.add('b', value=float(dict_F['X'+str(i)][1]))
            params.add('c', value=float(dict_F['X'+str(i)][2]))
            prx_1 = prx_1 + func_plin(pm_2['X'+str(i)] ,prx_1 , 'name_12' ,params['a'], params['b'], params['c'])
            prx_1 = prx_1.withColumn('result1' , f.col('result1') + prx_1['values'])

    return prx_1.withColumn('result' , exp(col('result') + col('result1')))

####################################################################################################

def model_func_cst(pm_EY_EST , PM , df ,funcid):
    pm_1= PM.dict_1
    dict_f = PM.dict_2
    pm_AY = PM.pm_AY
    pm_EY = pm_EY_EST + pm_AY

    cst = 0
    cst = df.withColumn('result' , cst + lit(pm_EY['XYZ_0'] + 0) + (col('name_1')+ lit(pm_EY['ABC_0'] + 0)) /lit(pm_EY['XYZ_5'] + 0) - lit(pm_EY['ABC_5'] + 0))
    cst = df.withColumn('result' , cst + lit(pm_EY['XYZ_1'] + 0) + (col('name_2')/ lit(pm_EY['ABC_1'] + 0)) /lit(pm_EY['XYZ_6'] + 0) - lit(pm_EY['ABC_6'] + 0))
    cst = df.withColumn('result' , cst + lit(pm_EY['XYZ_2'] + 0) + (col('name_3')/ lit(pm_EY['ABC_2'] + 0)) /lit(pm_EY['XYZ_7'] + 0) - lit(pm_EY['ABC_7'] + 0))
    cst = df.withColumn('result' , cst + lit(pm_EY['XYZ_3'] + 0) + (col('name_4')/ lit(pm_EY['ABC_3'] + 0)) /lit(pm_EY['XYZ_8'] + 0) - lit(pm_EY['ABC_8'] + 0))
    cst = df.withColumn('result' , cst + lit(pm_EY['XYZ_4'] + 0) + (col('name_5')/ lit(pm_EY['ABC_4'] + 0)) /lit(pm_EY['XYZ_9'] + 0) - lit(pm_EY['ABC_9'] + 0))
    cst = df.withColumn('result' , cst + lit(pm_EY['XYZ_10'] + 0) + (col('name_6')/ lit(pm_EY['ABC_15'] + 0)) /lit(pm_EY['ABC_10'] + 0) + lit(pm_EY['XYZ_15'] + 0))
    cst = df.withColumn('result' , cst + lit(pm_EY['XYZ_11'] + 0) + (col('name_7')/ lit(pm_EY['ABC_16'] + 0)) /lit(pm_EY['ABC_11'] + 0) + lit(pm_EY['XYZ_16'] + 0))
    cst = df.withColumn('result' , cst + lit(pm_EY['XYZ_12'] + 0) + (col('name_8')/ lit(pm_EY['ABC_17'] + 0)) /lit(pm_EY['ABC_12'] + 0) + lit(pm_EY['XYZ_17'] + 0))
    cst = df.withColumn('result' , cst + lit(pm_EY['XYZ_13'] + 0) + (col('name_9')/ lit(pm_EY['ABC_18'] + 0)) /lit(pm_EY['ABC_13'] + 0) + lit(pm_EY['XYZ_18'] + 0))
    cst = df.withColumn('result' , cst + lit(pm_EY['XYZ_14'] + 0) + (col('name_11')/ lit(pm_EY['ABC_19'] + 0)) /lit(pm_EY['ABC_14'] + 0) + lit(pm_EY['XYZ_19'] + 0))

    if funcid == 1:
      cst_1 = cst.withColumn('result1' , lit(0))
      for i in range(0 , 10):
        params = Parameters()
        params.add('a', value=float(dict_F['X'+str(i)][0]))
        params.add('b', value=float(dict_F['X'+str(i)][1]))
        params.add('c', value=float(dict_F['X'+str(i)][2]))
        cst_1 = func_plin(pm_2['X'+str(i)] ,cst_1 ,'name_12' ,params['a'], params['b'], params['c'])
        cst_1 = cst_1.withColumn('result1' , f.col('result1') + cst_1['values'])

    elif  funcid == 2:
        for i in range(10 , 20):
            params = Parameters()
            params.add('a', value=float(dict_F['X'+str(i)][0]))
            params.add('b', value=float(dict_F['X'+str(i)][1]))
            params.add('c', value=float(dict_F['X'+str(i)][2]))
            cst_1 = func_plin(pm_2['X'+str(i)] ,cst_1 , 'name_12' ,params['a'], params['b'], params['c'])
            cst_1 = cst_1.withColumn('result1' , f.col('result1') + cst_1['values'])

    return cst_1.withColumn('result' , exp(col('result') + col('result1')))

#############################################################################################

def model_func(pm_EY_EST , PM ,funcid , submod , df):
    if submod == 'TOT':
        y_mod = model_func_tot(pm_EY_EST , PM , df ,funcid )
    elif submod == 'PRX':
        y_mod = model_func_prx(pm_EY_EST , PM , df ,funcid )
    else:
        y_mod = model_func_cst(pm_EY_EST , PM , df ,funcid )
    return  y_mod

####################################################################
def model_func_fit(pm_EY_EST , PM  ,funcid , submod, df):
    y_mod = model_func(pm_EY_EST , PM  ,funcid , submod, df)
    er = y_mod.withColumn('er' , col('result') - df['Y_act'])
    er = er.withColumn('er' , col('er') * sqrt(abs(col('wgt'))))
    er = np.array([row.er for row in er.select('er').collect()])
    return er

In [ ]:
import datetime
start_time = datetime.datetime.now()
print ("Time start time is  : ", end = "")
print (start_time)

#############################################

pm_EY_EST = PM.pm_EY
funcid = 1
submod = 'TOT'
out = Minimizer(model_func_fit, pm_EY_EST  ,fcn_args = ( PM  ,funcid , submod,df_data,))
out = out.minimize()

###########################################
end_time = datetime.datetime.now()
print ("Time end time is  : ", end = "")
print (end_time)

Time start time is  : 2023-12-19 08:30:35.070658
Time end time is  : 2023-12-19 08:31:10.913418


In [ ]:
out